<h1><center>AUTOMATING POST PULL REQUESTS</center></h1>

<img src = './visuals/aita.png'>
<a href= 'https://www.reddit.com/r/AmItheAsshole/'> Am I The Asshole </a>
<img src = './visuals/jnmil.png'>
<a href= 'https://www.reddit.com/r/JUSTNOMIL/'> Just NO Mother-In-Law </a>

In [1]:
# my needed imports
import pandas as pd   
import requests 
import time   
import datetime

## Creating a function to automate pull requests from PushShift API for my 2 chosen subreddits

The function will:
- make a url request to pushshift using the subreddit of choice using parameters indicated
- even if I put n_requests to 5000, the function will pull 100, wait for 30 seconds and run next 100
    - based on timestamp on last pulled post, it will not repeat same posts
    
- A warning will be printed if :
    - The status code is anything other than 200
    - There is a connection error of unknown reason
    - The pull request did not add anything to our new_list (nothing pulled / to pull)
    
- As the function is running it will inform mer:
    - Which pull request it is on
    - How many it pulled that request
    - And what the total length of our big list is currently at
    
- When all requests have been pulled the posts will be:
    - Saved to a dataframe, with columns: Author, title, subtext, subreddit
    - The new dataframe will be saved to csv to be used later.
    


In [2]:
posts = [] # empty posts list outside function so I can access later
    
def reddit_pull(subreddit, n_requests):
    
    url = 'https://api.pushshift.io/reddit/search/submission/' # base url
    last_obtained_time = round(time.time()) # will be updated with the timestamp of last aquired post
    # initially I had it set to 0, but it wouldnt' get past the first if clause below. maybe because 0 is not a time?
    # got time.time() from w3


    pull = 1 # since we can only pull certain amounts in a 30 second timeframe, I will start with pull 1 and increment
   
    while len(posts) < n_requests: # while the length of list 'posts' is less than the number of requests I made
        
        try: # I'm going to do a try except to make sure I get notice if there is a connection error
            
            search_params = {
                'subreddit': subreddit, # the subreddit of choice
                'sort':'desc', # sort results in descending format
                'size': n_requests,  # the size of the request total
                'before': last_obtained_time-1,
                'user_removed' : [True],
                'mod_removed' : [True],
                                 }
            

            r = requests.get(url, params = search_params) # make the request using the params set
            data = r.json() # make the data into .json
            new_posts = data['data'] # get the dictionary 'data'
            
            
            if len(new_posts) == 0: # if the length of the new aquired posts is equal to 0
                last_obtained_time = last_obtained_time # the last obtained time remains the same
                # Do nothing else, don't extend the list, don't increase the pull
                print('This pull request has not pulled any new posts')
                
            else:     
                last_obtained_time = new_posts[-1]['created_utc'] #otherwise change the time to the last aquired post's time
                posts.extend(new_posts) # extend adds all elements gathered from new_posts to the posts list, whereas .append will
                                #take all elements gathered as 1 and add it to list
                pulled = len(new_posts)        # which will tell me how many pulls it has made so far and how many posts in that pull
                print(f'Pull Request {pull} has pulled {pulled} submissions')
                print(f'There are now {len(posts)} total posts') # Lets me know which pull request it is and how many were pulled.
                time.sleep(30) # sleep for 30 seconds before next pull
                pull += 1 # adds 1 to pull counter to indicate it is the next pull
                

        
               
        # The except part, to throw up error if connection
        except:
            if r.status_code != 200: # if the status code is anything but 200
                print(f'Connection Error Code: {r.status_code}')
            else: # otherwise if the error is not status related:
                print('Could not complete pull, Reason Unknown')
                
        ## Saving to DF and to csv for later use
        df = pd.DataFrame(posts)[['author', 'title', 'selftext', 'subreddit']]
        df.to_csv('./data/' + subreddit + '.csv')    
  

In [3]:
#reddit_pull('AmItheAsshole', 9000) # 9000 because out of the original 5000 I pulled 1700 were removed
                                    # even through I added filter in search params some still showed up
                                    # 9000 will make sure I still have more than 5000

An Example of the print statements as function runs:
- Pull Request 1 has pulled 100 submissions
    - There are now 100 total posts
- Pull Request 2 has pulled 100 submissions
    - There are now 200 total posts

In [4]:
# make a df out of pulled data list
# reddit_pull('justnomil', 7000)

The acquired posts are now saved into a csv to be used in next notebook (Data Cleanup & EDA)